# Random Forests and Decision Trees
- Classify the Spotify.csv dataset 
- Predict the Genre 
- The dataset has 23 Columns and The Output feature is **playlist_genre** so all others can be used as input features. Decide on what can be used and what cannot be used. 
- Do the Preprocessing below
- Do 70:30 Train test split 

In [26]:
import pandas as pd 
data = pd.read_csv('spotify Edited.csv')
data.head()

,track_id,track_name,track_artist,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,playlist_genre,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,6f807x0ima9a1j3VPbc7VN,I Don't Care (with Justin Bieber) - Loud Luxur...,Ed Sheeran,66,2oCs0DGTsRO98Gh5ZSl2Cx,I Don't Care (with Justin Bieber) [Loud Luxury...,2019-06-14,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,6,-2.634,1,0.0583,0.1020,0.000000,0.0653,0.518,122.036,194754
1,0r7CVbZTWZgbTCYdfa2P31,Memories - Dillon Francis Remix,Maroon 5,67,63rPSO264uRjW1X5E6cWv6,Memories (Dillon Francis Remix),2019-12-13,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,11,-4.969,1,0.0373,0.0724,0.004210,0.3570,0.693,99.972,162600
2,1z1Hg7Vb0AhHDiEmnDE79l,All the Time - Don Diablo Remix,Zara Larsson,70,1HoSmj2eLcsrR0vE9gThr4,All the Time (Don Diablo Remix),2019-07-05,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,1,-3.432,0,0.0742,0.0794,0.000023,0.1100,0.613,124.008,176616
3,75FpbthrwQmzHlBJLuGdC7,Call You Mine - Keanu Silva Remix,The Chainsmokers,60,1nqYsOef1yKKuGOVchbsk6,Call You Mine - The Remixes,2019-07-19,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,7,-3.778,1,0.1020,0.0287,0.000009,0.2040,0.277,121.956,169093
4,1e8PAfcKUYoKkxPhrHqw4x,Someone You Loved - Future Humans Remix,Lewis Capaldi,69,7m7vv9wlQ4i0LFuJiE2zsQ,Someone You Loved (Future Humans Remix),2019-03-05,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,1,-4.672,1,0.0359,0.0803,0.000000,0.0833,0.725,123.976,189052


In [27]:
data.dropna(inplace=True)

In [28]:
ColsToDrop = ['track_id', 'track_name', 'track_album_id',
              'track_album_name', 'track_album_release_date']

data.drop(columns=ColsToDrop, inplace=True)

categoricalCols = data.select_dtypes(include=object).columns
numericalCols = data.select_dtypes(include=['int64', 'float64']).columns

dataCategorical = data[categoricalCols]
dataNumerical = data[numericalCols]

In [48]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

for col in dataCategorical.columns:
    dataCategorical.loc[:, col] = encoder.fit_transform(dataCategorical[col])

In [49]:
dataCategorical.head()

,track_artist,playlist_name,playlist_id,playlist_genre,playlist_subgenre
0,2782,292,235,2,3
1,6084,292,235,2,3
2,10416,292,235,2,3
3,9215,292,235,2,3
4,5402,292,235,2,3


In [21]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaledNumericalData = scaler.fit_transform(dataNumerical)
scaledNumericalData = pd.DataFrame(scaledNumericalData, columns=dataNumerical.columns)

In [55]:
scaledNumericalData.head()

,track_popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,0.941407,0.642015,1.201668,0.173348,1.367040,0.876132,-0.481253,-0.333961,-0.377985,-0.809246,0.031929,0.042833,-0.518802
1,0.981439,0.490384,0.643388,1.557808,0.585736,0.876132,-0.688547,-0.468726,-0.359210,1.081094,0.782525,-0.777291,-1.056175
2,1.101535,0.138877,1.284580,-1.211111,1.100025,-1.141381,-0.324301,-0.436856,-0.377881,-0.519571,0.439395,0.116133,-0.821933
3,0.701216,0.435246,1.279053,0.450240,0.984251,0.876132,-0.049884,-0.667687,-0.377943,0.089589,-1.001750,0.039860,-0.947661
4,1.061503,-0.033430,0.742884,-1.211111,0.685114,0.876132,-0.702366,-0.432758,-0.377985,-0.692598,0.919777,0.114944,-0.614097


In [56]:
if not scaledNumericalData.index.equals(dataCategorical.index):
    scaledNumericalData = scaledNumericalData.reset_index(drop=True)
    dataCategorical = dataCategorical.reset_index(drop=True)

dataModified = pd.concat([scaledNumericalData, dataCategorical], axis=1)
dataModified.head()

,track_popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,track_artist,playlist_name,playlist_id,playlist_genre,playlist_subgenre
0,0.941407,0.642015,1.201668,0.173348,1.367040,0.876132,-0.481253,-0.333961,-0.377985,-0.809246,0.031929,0.042833,-0.518802,2782,292,235,2,3
1,0.981439,0.490384,0.643388,1.557808,0.585736,0.876132,-0.688547,-0.468726,-0.359210,1.081094,0.782525,-0.777291,-1.056175,6084,292,235,2,3
2,1.101535,0.138877,1.284580,-1.211111,1.100025,-1.141381,-0.324301,-0.436856,-0.377881,-0.519571,0.439395,0.116133,-0.821933,10416,292,235,2,3
3,0.701216,0.435246,1.279053,0.450240,0.984251,0.876132,-0.049884,-0.667687,-0.377943,0.089589,-1.001750,0.039860,-0.947661,9215,292,235,2,3
4,1.061503,-0.033430,0.742884,-1.211111,0.685114,0.876132,-0.702366,-0.432758,-0.377985,-0.692598,0.919777,0.114944,-0.614097,5402,292,235,2,3


In [59]:
y = dataModified.pop('playlist_genre').astype(int)
X = dataModified

In [68]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## Frame a Decision Tree Model 
- Use Any Parameters
- You can use any library

In [74]:
from sklearn.tree import DecisionTreeClassifier

dtModel = DecisionTreeClassifier(max_depth=100, random_state=42)

## Frame a Random Forests Model 
- Use Any Parameters
- You can use any library

In [75]:
from sklearn.ensemble import RandomForestClassifier

rfModel = RandomForestClassifier(n_estimators=50, max_depth=50, random_state=42)

## Train the Models and Report the Accuracy

In [76]:
dtModel.fit(X_train, y_train)
y_pred_DT = dtModel.predict(X_test)

rfModel.fit(X_train, y_train)
y_pred_RF = dtModel.predict(X_test)

In [77]:
from sklearn.metrics import accuracy_score
print(f"DECISION TREE ACCURACY = {accuracy_score(y_true=y_test, y_pred=y_pred_DT)}")
print(f"RANDOM FOREST ACCURACY = {accuracy_score(y_true=y_test, y_pred=y_pred_RF)}")

DECISION TREE ACCURACY = 0.9990862016448371
RANDOM FOREST ACCURACY = 0.9990862016448371
